In [3]:
!pip install requests
!pip install beautifulsoup4



In [4]:
import requests
from bs4 import BeautifulSoup

def scrape_docs(url):
    """Scrape text content from the provided URL."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')  # Adjust as needed
    return " ".join([p.get_text() for p in paragraphs])

# URLs to scrape
urls = [
    "https://segment.com/docs/?ref=nav",
    "https://docs.mparticle.com/",
    "https://docs.lytics.com/",
    "https://docs.zeotap.com/home/en-us/",
]

# Scrape all documentation
all_docs = []
for url in urls:
    all_docs.append(scrape_docs(url))

# Save to a file for later use
with open("all_docs.txt", "w") as f:
    f.write("\n".join(all_docs))


In [5]:
def chunk_text(text, chunk_size=500):
    """Split text into chunks of the specified size."""
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Load and chunk all scraped documents
with open("all_docs.txt", "r") as f:
    docs = f.read()

text_chunks = chunk_text(docs)


In [6]:
pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.7 MB/s eta 0:00:00


In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create FAISS index
dimension = 384  # Dimension of 'all-MiniLM-L6-v2'
index = faiss.IndexFlatL2(dimension)

# Generate embeddings and store in FAISS
embeddings = [embedding_model.encode(chunk) for chunk in text_chunks]
index.add(np.array(embeddings, dtype="float32"))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def retrieve_relevant_context(query, index, text_chunks):
    """Retrieve the most relevant chunks for a query."""
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k=5)
    return [text_chunks[i] for i in indices[0]]


In [9]:
pip install transformers


In [10]:
from transformers import pipeline

# Initialize text generation model
response_model = pipeline("text-generation", model="gpt2")

def generate_response(context, query):
    """Generate a response based on context and query."""
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = response_model(prompt, max_length=200, num_return_sequences=1)
    return response[0]["generated_text"]


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [11]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [12]:
import streamlit as st

st.title("CDP How-To Chatbot")
st.markdown("Ask 'how-to' questions about Segment, mParticle, Lytics, and Zeotap.")

query = st.text_input("Enter your question:", placeholder="E.g., How do I set up a new source in Segment?")

if st.button("Get Answer"):
    if query:
        with st.spinner("Fetching answer..."):
            relevant_context = retrieve_relevant_context(query, index, text_chunks)
            combined_context = " ".join(relevant_context)
            response = generate_response(combined_context, query)
        st.markdown(f"**Answer:** {response}")
    else:
        st.warning("Please enter a question.")


2025-01-13 18:59:14.082 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.698 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-13 18:59:14.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.732 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.735 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 18:59:14.741 Thread 'MainThread': mi

In [13]:
from sentence_transformers import SentenceTransformer, util

def retrieve_relevant_context(query, index, text_chunks, top_k=3):
    """
    Retrieve the most relevant context from text chunks based on the query.
    :param query: The user query as a string.
    :param index: The vector index (e.g., a FAISS or other retrieval model).
    :param text_chunks: List of text chunks corresponding to the index.
    :param top_k: Number of top results to return.
    :return: List of relevant text chunks.
    """
    # Convert the query into an embedding
    query_embedding = index.encode([query], convert_to_tensor=True)

    # Perform similarity search
    scores = util.pytorch_cos_sim(query_embedding, index)
    top_results = scores.topk(top_k)

    relevant_chunks = [text_chunks[i] for i in top_results.indices]
    return relevant_chunks


In [15]:
from sentence_transformers import SentenceTransformer, util

# Instantiate model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def retrieve_relevant_context(query, index, text_chunks, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, index)
    top_results = scores.topk(top_k)
    relevant_chunks = [text_chunks[i] for i in top_results.indices]
    return relevant_chunks



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
from sentence_transformers import SentenceTransformer, util

# Instantiate model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def retrieve_relevant_context(query, index, text_chunks, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, index)
    top_results = scores.topk(top_k)
    relevant_chunks = [text_chunks[i] for i in top_results.indices]
    return relevant_chunks


In [18]:
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to retrieve relevant context
def retrieve_relevant_context(query, model, text_chunks, top_k=3):
    # Generate the embedding for the query
    query_embedding = model.encode([query], convert_to_tensor=True)

    # Create the embeddings for the text chunks
    text_embeddings = model.encode(text_chunks, convert_to_tensor=True)

    # Calculate cosine similarity between the query and text chunks
    scores = util.pytorch_cos_sim(query_embedding, text_embeddings)

    # Get the top-k results
    top_results = scores.topk(top_k)

    # Retrieve the corresponding text chunks
    relevant_chunks = [text_chunks[i] for i in top_results.indices[0]]

    return relevant_chunks

# Example usage
query = "How do I set up a new source in Segment?"
text_chunks = [
    "To set up a new source in Segment, go to the Sources tab.",
    "You can integrate various platforms with Segment by adding a new source.",
    "First, log into Segment and click on 'Add Source' to begin the setup process.",
    "Setting up Segment involves configuring your API keys and sources."
]

# Retrieve the relevant context
relevant_context = retrieve_relevant_context(query, model, text_chunks)

# Print the relevant context
print(relevant_context)



['To set up a new source in Segment, go to the Sources tab.', 'You can integrate various platforms with Segment by adding a new source.', 'Setting up Segment involves configuring your API keys and sources.']
